In [ ]:
## Project Description 

In previous week, we have explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. 
One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. 
Is New York City more like Toronto or Paris or some other multicultural city? 
In order to solve these questions, we will leverage the Foursquare location data to explore or compare neighborhoods. 

# 1.1 Background

New York City (NYC), often called simply New York, is the most populous city in the United States. 
With an estimated 2019 population of 8,336,817 distributed over about 302.6 square miles, 
New York City is also the most densely populated major city in the United States.
Located at the southern tip of the U.S. state of New York, the city is the center of the New York metropolitan area, 
the largest metropolitan area in the world by urban landmass.With almost 20 million people in its metropolitan statistical area
and approximately 23 million in its combined statistical area, it is one of the world's most populous megacities.

Situated on one of the world's largest natural harbors, New York City is composed of five boroughs, 
each of which is a county of the State of New York. The five boroughs—Brooklyn, Queens, Manhattan, the Bronx, and Staten.
The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. 
As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. 
New York is home to more than 3.2 million residents born outside the United States, the largest foreign-born population of any city in the world as of 2016.
As of 2019, the New York metropolitan area is estimated to produce a gross metropolitan product (GMP) of $2.0 trillion. 

(All info from https://en.wikipedia.org/wiki/New_York_City)

# 1.2 Related Questions

Based on the above info, we will answer three questions below:
    1. What the most visited venus in New York City?
    2. Which neighborhodd has the most venues? 
    3. What the best investment choices in new york city if someone want to open a restaurant?

By answering above three questions, it will help tourists or investors to know what place is better to visit, invest or to start their own business based 
on the venues information. This will slso helps people to make decisions if they are thinking about migrating to New York City. 



In [ ]:
Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [1]:
%pip install jupyter_contrib_nbextensions --user

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install geocoder --user


You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install lxml --user


You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install beautifulsoup4 --user


You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install folium --user


You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


/Users/ethan/Library/Python/3.7/lib/python/site-packages/scipy/__init__.py:137: UserWarning: NumPy 1.16.5 or above is required for this version of SciPy (detected version 1.16.2)
  UserWarning)


Libraries imported.


In [6]:
#Foursquare info 

CLIENT_ID = '3DEXXP0RNJU15WJ1EHOX2DGQ0KZTVBQ4YT0VTTN5GG3LDVPK' # your Foursquare ID
CLIENT_SECRET = 'BPTILR41OA4T4QNQMM0CGWAPZ03SH1QB1EYODS5AW3S5JJK4' # your Foursquare Secret
ACCESS_TOKEN = 'K2HLB4IYZV10JDOAUXQQHET234BDLWCZNVA2GUTF3HITOYZD' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3DEXXP0RNJU15WJ1EHOX2DGQ0KZTVBQ4YT0VTTN5GG3LDVPK
CLIENT_SECRET:BPTILR41OA4T4QNQMM0CGWAPZ03SH1QB1EYODS5AW3S5JJK4


In [7]:
with open('/Users/ethan/Downloads/Week4_Project/nyu.json') as json_data:
    ny_data = json.load(json_data)

In [10]:
neighborhoods_data = ny_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_geo = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_geo[1]
    neighborhood_log = neighborhood_geo[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_log}, ignore_index = True)

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [15]:
map_ny = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

In [42]:
### use foursquare api to explore 
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                            latitudes=neighborhoods['Latitude'],
                            longitudes=neighborhoods['Longitude'])

In [18]:
print(ny_venues.shape)


(10121, 7)


In [19]:
ny_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
5,Wakefield,40.894705,-73.847201,Subway,40.890468,-73.849152,Sandwich Place
6,Wakefield,40.894705,-73.847201,Central Deli,40.896728,-73.844387,Deli / Bodega
7,Wakefield,40.894705,-73.847201,Koss Quick Wash,40.891281,-73.849904,Laundromat
8,Co-op City,40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place
9,Co-op City,40.874294,-73.829939,Rite Aid,40.870345,-73.828302,Pharmacy


In [21]:
# numbers of venues in each neighborhood
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,30,30,30,30,30,30
Annadale,11,11,11,11,11,11
Arden Heights,5,5,5,5,5,5
Arlington,5,5,5,5,5,5
Arrochar,23,23,23,23,23,23
Arverne,20,20,20,20,20,20
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,18,18,18,18,18,18


In [22]:
# count how many uniques categories 
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 429 uniques categories.


In [23]:
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Offi

In [24]:
ny_onehot.shape


(10121, 429)

In [25]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()

ny_grouped.shape

(302, 429)

In [26]:
## Top 5 most common venues
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                 venue  freq
0          Pizza Place  0.17
1        Deli / Bodega  0.07
2          Supermarket  0.07
3   Chinese Restaurant  0.07
4  Rental Car Location  0.03


----Annadale----
          venue  freq
0   Pizza Place  0.18
1    Restaurant  0.09
2  Liquor Store  0.09
3      Pharmacy  0.09
4          Park  0.09


----Arden Heights----
              venue  freq
0          Pharmacy   0.2
1     Deli / Bodega   0.2
2       Coffee Shop   0.2
3  Business Service   0.2
4       Pizza Place   0.2


----Arlington----
                 venue  freq
0             Bus Stop   0.4
1        Deli / Bodega   0.2
2         Intersection   0.2
3  American Restaurant   0.2
4          Yoga Studio   0.0


----Arrochar----
                venue  freq
0            Bus Stop  0.13
1       Deli / Bodega  0.09
2          Bagel Shop  0.09
3  Italian Restaurant  0.09
4         Pizza Place  0.09


----Arverne----
            venue  freq
0       Surf Spot  0.20
1   Metro Station  0.10
2  Sand

In [27]:
# sort the venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
## new dataframe with top 5 venues
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Chinese Restaurant,Deli / Bodega,Spanish Restaurant
1,Annadale,Pizza Place,Sushi Restaurant,Bakery,Liquor Store,Pharmacy
2,Arden Heights,Deli / Bodega,Pharmacy,Coffee Shop,Business Service,Pizza Place
3,Arlington,Bus Stop,Intersection,Deli / Bodega,American Restaurant,Women's Store
4,Arrochar,Bus Stop,Italian Restaurant,Pizza Place,Deli / Bodega,Liquor Store


In [29]:
## divide into 5 clusters

# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 0, 0, 0, 0, 0, 0, 4], dtype=int32)

In [30]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = neighborhoods
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4,Pharmacy,Deli / Bodega,Donut Shop,Sandwich Place,Dessert Shop
1,Bronx,Co-op City,40.874294,-73.829939,4,Fast Food Restaurant,Bus Station,Pizza Place,Post Office,Basketball Court
2,Bronx,Eastchester,40.887556,-73.827806,4,Caribbean Restaurant,Bus Station,Diner,Deli / Bodega,Platform
3,Bronx,Fieldston,40.895437,-73.905643,0,Medical Supply Store,Bus Station,River,Plaza,Women's Store
4,Bronx,Riverdale,40.890834,-73.912585,0,Park,Bank,Bus Station,Gym,Playground


In [35]:
## show the results

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
## Map of the clusters 

In [ ]:
# Cluster 1 

In [36]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Fieldston,Medical Supply Store,Bus Station,River,Plaza,Women's Store
4,Riverdale,Park,Bank,Bus Station,Gym,Playground
9,Williamsbridge,Bar,Nightclub,Caribbean Restaurant,Soup Place,Fast Food Restaurant
12,City Island,Seafood Restaurant,Thrift / Vintage Store,Italian Restaurant,Arts & Crafts Store,Spanish Restaurant
18,West Farms,Bus Station,Park,Donut Shop,Playground,Cuban Restaurant
22,Port Morris,Furniture / Home Store,Bar,Distillery,Spanish Restaurant,Brewery
28,Throgs Neck,Italian Restaurant,Pizza Place,Asian Restaurant,Baseball Field,Deli / Bodega
29,Country Club,Sandwich Place,Italian Restaurant,Flea Market,Playground,Athletics & Sports
39,Edgewater Park,Italian Restaurant,Deli / Bodega,Pizza Place,Coffee Shop,Asian Restaurant
41,Olinville,Caribbean Restaurant,Supermarket,Fried Chicken Joint,Convenience Store,Deli / Bodega


In [37]:
# Cluster 2 
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
255,Emerson Hill,Food,Women's Store,Fast Food Restaurant,Entertainment Service,Ethiopian Restaurant


In [38]:
# Cluster 3 
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
257,Howland Hook,Boat or Ferry,Women's Store,Filipino Restaurant,Ethiopian Restaurant,Event Service


In [39]:
# Cluster 4 
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
27,Clason Point,Park,Boat or Ferry,Moving Target,Convenience Store,Pool
192,Somerville,Park,Women's Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant
203,Todt Hill,Park,Trail,Women's Store,Farmers Market,Entertainment Service


In [40]:
# Cluster 5 
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Wakefield,Pharmacy,Deli / Bodega,Donut Shop,Sandwich Place,Dessert Shop
1,Co-op City,Fast Food Restaurant,Bus Station,Pizza Place,Post Office,Basketball Court
2,Eastchester,Caribbean Restaurant,Bus Station,Diner,Deli / Bodega,Platform
5,Kingsbridge,Pizza Place,Sandwich Place,Bar,Mexican Restaurant,Latin American Restaurant
6,Marble Hill,Sandwich Place,Coffee Shop,Gym,Discount Store,Yoga Studio
7,Woodlawn,Pub,Pizza Place,Deli / Bodega,Playground,Italian Restaurant
8,Norwood,Pizza Place,Park,Bank,Pharmacy,Fast Food Restaurant
10,Baychester,Donut Shop,Fried Chicken Joint,Discount Store,Bus Station,Supermarket
11,Pelham Parkway,Italian Restaurant,Frozen Yogurt Shop,Pizza Place,Café,Bank
13,Bedford Park,Mexican Restaurant,Chinese Restaurant,Diner,Deli / Bodega,Pizza Place


In [ ]:
###
In conclusion, overall new york city is a diversity city. Based on the data above, 
Italian Restaturant, Coffee shop, Pizza Place are the most common resutarnts in the city. 
On the other hand, there are few interesting things:
    1. In neighborhood 'Wingate','Holliswood', and 'New Lots' , it seems like people really love fried chicken 
    2. Boba tea shop is the most common venue in neighborhood 'Whitestone'
    3. Indian restaurant is the most common resutarnt in neighborhood 'Floral Park'